In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from warnings import simplefilter,filterwarnings
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from matplotlib.colors import ListedColormap
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from matplotlib.ticker import MaxNLocator
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
import matplotlib.ticker as mticker
import matplotlib.ticker as ticker
from sklearn.utils import shuffle
from contextlib import suppress
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from random import shuffle
import seaborn as sns
from time import time
import pandas as pd
import numpy as np
import scipy.io
import pickle
import glob
import sys
import gc
import os
import re
filterwarnings('ignore')
filterwarnings(action='once')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
simplefilter('ignore', FutureWarning)

In [31]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
code_path = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/Codes/Shared Codes'
data_path = '/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/data'
results_path = "/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/Results"
image_path1 = "/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/Images"
# CSV_path = "/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/CSV"
image_path = "/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/article 2"
CSV_path = "/content/drive/MyDrive/Colab_Notebooks/Muawiya/Genetic Programming Combiner with DFS/article 2"

# folders_evolving = ["kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest",
#                     "kpfaoselm_gp_combiner_with_model_replacement_evolving_DFS",
#                     # "kpfaoselm_gp_combiner_with_model_replacement_evolving",
#                     "kpfaoselm_gp_combiner_with_model_replacement_evolving_IssamDFS",
#                     ]
# folders_without_evolving = ["kpfaoselm_gp_combiner_with_model_replacement_random_forest",
#                             "kpfaoselm_gp_combiner_with_model_replacement_DFS",
#                             # "kpfaoselm_gp_combiner_with_model_replacement",
#                             "kpfaoselm_gp_combiner_with_model_replacement_IssamDFS",
#                             ]
# algo_name = 'kpfaoselm_gp'

folders_evolving = ["faoselm_gp_combiner_with_model_replacement_evolving_random_forest",
                    "faoselm_gp_combiner_with_model_replacement_evolving_DFS",
                    # "faoselm_gp_combiner_with_model_replacement_evolving",
                    "faoselm_gp_combiner_with_model_replacement_evolving_IssamDFS",
                    ]
folders_without_evolving = ["faoselm_gp_combiner_with_model_replacement_random_forest",
                            "faoselm_gp_combiner_with_model_replacement_DFS",
                            # "faoselm_gp_combiner_with_model_replacement",
                            "faoselm_gp_combiner_with_model_replacement_IssamDFS",
                            ]
algo_name = 'faoselm_gp'


# folders_evolving = ["gp_combiner_with_model_replacement_evolving_random_forest",
#                     "gp_combiner_with_model_replacement_evolving_DFS",
#                     # "gp_combiner_with_model_replacement_evolving",
#                     "gp_combiner_with_model_replacement_evolving_IssamDFS",
#                     ]
# folders_without_evolving = ["gp_combiner_with_model_replacement_random_forest",
#                             "gp_combiner_with_model_replacement_DFS",
#                             # "gp_combiner_with_model_replacement",
#                             "gp_combiner_with_model_replacement_IssamDFS",
#                             ]
# algo_name = 'gp'



sys.path.insert(0,code_path)
from ensemble import Ensemble, Classifier

In [33]:
filenames = ['ISCX2012.csv', 'CSE-CIC2018.csv', 'CICIDS2017.csv', 'kddcup99_csv.csv', 'Blip_Drift.csv',
             'Gradual_Drift.csv', 'Incremental_Drift.csv', 'Recurrent_Drift.csv', 'Sudden_Drift.csv']
files = list(map(lambda x: os.path.join(data_path, x), filenames))

In [34]:
def save_object(obj, filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()
def load_object(filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object
def save_pickle(obj, file_name):
  with open(file_name, 'wb') as f:
    pickle.dump(obj, f)
def load_pickle(file_name):
  with open(file_name, 'rb') as f:
    d = pickle.load(f)
  return d

In [35]:
def plot_results(metric_name, methods_name, results, colors, data_name, algo_name, drift_location=None, Name="", step=1):
    plt.figure(figsize=(25, 8))
    plt.title(data_name+" "+Name)
    for k, result in enumerate(results):
        x = [i for i, x in enumerate(results[k]) if i % step == 0]
        y = [x for i, x in enumerate(results[k]) if i % step == 0]
        plt.plot(x, y, color=colors[k], label=methods_name[k])
        plt.scatter(x, y, color=colors[k], s=20)
        if not drift_location is None:
          for loc in drift_location[k]:
            if type(loc) is str : continue
            loc = loc-1
            y_value = y[loc]
            plt.plot(loc, y_value, marker="o", markersize=9, markerfacecolor=colors[k])
    plt.legend(loc="best")
    plt.xlabel('Chunk number')
    plt.ylabel(metric_name)
    ax = plt.gca()
    ax.locator_params(axis='y', nbins=15)
    ax.yaxis.set_major_locator(MaxNLocator(nbins=15))
    num_ticks = 50
    ax.xaxis.set_major_locator(plt.MaxNLocator(num_ticks))
    ax.xaxis.set_major_formatter(plt.ScalarFormatter(useOffset=False, useMathText=True))
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
    plt.xticks(rotation=45, fontsize='xx-small')
    plt.grid()
    my_path = os.path.join(image_path,"{}".format(data_name))
    os.makedirs(my_path, exist_ok=True)
    plt.savefig(os.path.join(my_path, algo_name+"_"+metric_name+"_"+Name))
    plt.savefig(os.path.join(my_path, algo_name+"_"+metric_name+"_"+Name+'.svg'), format='svg')
    plt.show()

In [36]:
def Finall_Results(folders,Name=""):
  synthetic_data = ['Blip', 'GradualDrift', 'Incremental', 'Recurrent', 'SuddenDrift']

  data_names = ['ISCX2012', 'CSE-CIC2018', 'CICIDS2017', 'KddCup99', 'Blip',
                'GradualDrift', 'Incremental', 'Recurrent', 'SuddenDrift']

  # method_names = ["GPC-KOS-RF","GPC-KOS-DFS", "GPC-KOS-DQN-MAFS"] #  # in index 2 "GPC-KOS",
  method_names = ["GPC-FOS-RF", "GPC-FOS-DFS",  "GPC-FOS-DQN-MAFS"] #  # in index 2 "GPC-FOS",
  # method_names = ["GPC-OS-RF", "GPC-OS-DFS",  "GPC-OS-DQN-MAFS"] #  # in index 2 "GPC-OS",

  # metrices = ["accuracy", "precision", "recall", "f1-score", "auc", "Prediction Times", "Memory Usage"]
  metrices = ["Memory Usage"]
  new_metrices = ['Selected features']
  # df_accuracy, df_precision , df_recall , df_f1_score, df_auc, df_drift_location = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

  # df_accuracy['data_names'],df_precision['data_names'] =  data_names, data_names
  # df_f1_score['data_names'],df_recall['data_names'] = data_names, data_names
  # df_auc['data_names'], df_drift_location['data_names'] = data_names, data_names

  # overall_results = {"accuracy":df_accuracy, "precision":df_precision, "recall":df_recall, "f1-score":df_f1_score, "auc":df_auc}

  colors = ['g', 'r', 'b','orange']
  data_folders = list(map(lambda x: os.path.join(image_path1, x), data_names))
  for i, data_folder in enumerate(data_folders):
    # accuracys, precisions, recalls, f1_scores, aucs, drift_location, memory_reduction, prediction_times = [], [], [], [], [], [], [], []
    for j, metrice in enumerate(metrices):
      csv_path = os.path.join(data_folder, algo_name+"_"+metrice+"_"+Name)
      new_csv_path = os.path.join(data_folder, algo_name+"_"+new_metrices[j]+"_"+Name)

      df = pd.read_csv(csv_path+".csv")
      # df = df.rename(columns={'GPC-KOS-RF': 'GPC-KOS-RF', 'GPC-KOS-DFS': 'GPC-KOS-DFS', 'GPC-OS': 'GPC-KOS', 'GPC-KOS-DQN-MAFS': 'GPC-KOS-DQN-MAFS'})

      # df = df.rename(columns={'GPC-KOS-RF': 'GPC-FOS-RF', 'GPC-KOS-DFS': 'GPC-FOS-DFS', 'GPC-OS': 'GPC-FOS', 'GPC-KOS-DQN-MAFS': 'GPC-FOS-DQN-MAFS'})

      # if list(df.columns) == ['GPC-FOS-RF', 'GPC-FOS-DFS' ,'GPC-FOS' ,'GPC-FOS-DQN-MAFS']:
      #   df = df.rename(columns={'GPC-FOS-RF': 'GPC-OS-RF', 'GPC-FOS-DFS': 'GPC-OS-DFS', 'GPC-FOS': 'GPC-OS', 'GPC-FOS-DQN-MAFS': 'GPC-OS-DQN-MAFS'})
      # else:
      #  df = df.rename(columns={'GPC-KOS-RF': 'GPC-OS-RF', 'GPC-KOS-DFS': 'GPC-OS-DFS', 'GPC-OS': 'GPC-OS', 'GPC-KOS-DQN-MAFS': 'GPC-OS-DQN-MAFS'})


      df.to_csv(os.path.join(new_csv_path+".csv"), index=False)
      results = []
      drift_location = []
      for k, method_name in enumerate(method_names):
        temp = df[method_name].tolist()
      #   # if not metrice in ["Prediction Times", "Memory Usage"]:
      #   #   overall_results[metrice].at[i,method_name] = np.average(temp)
      #   # np.average(temp)
        results.append(temp)
        position = df.columns.get_loc(method_name) + 1
        vector = np.zeros(len(df[method_name].tolist()))
        if data_names[i] in synthetic_data:
          drift_location.append([25,50,75])
          vector[[25,50,75]] = 1
          if not 'drift_location_'+method_name in list(df.columns):
            df.insert(position, 'drift_location_'+method_name, vector)
        else:
          locations = list(load_object(filenames[i]+"_drift_location",os.path.join(os.path.join(results_path,folders[k]),filenames[i])).keys())
          locations = np.array(locations) - 1
          if not len(locations) == 0:
            vector[locations] = 1
          drift_location.append(locations)
          if not 'drift_location_'+method_name in list(df.columns):
            df.insert(position, 'drift_location_'+method_name, vector)
      df.to_csv(os.path.join(new_csv_path+".csv"), index=False)
      plot_results(new_metrices[j], method_names, results, colors, data_names[i], algo_name, drift_location, Name, step=1)

In [37]:
Finall_Results(folders=folders_evolving,Name="With Evolving")

Output hidden; open in https://colab.research.google.com to view.

In [38]:
Finall_Results(folders=folders_without_evolving,Name="Without Evolving")

Output hidden; open in https://colab.research.google.com to view.

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
